In [1]:
from pymoab.core import Core  # meshdb
import pymoab.types as mt # moab types
from pydtk2 import *  # mapper
from ovcg import *  # fluid solver
from fesol import *  # FEM solver
from mpi4py import MPI  # fix openmpi with python
import numpy as np
from meshio.gmsh_io import read

In [2]:
# retrieve the interface grids

g_f = Grids('inputs/fluid_quad2p.hdf')
grid1 = 0
grid2 = 1
indices1 = g_f.get_boundary_indices(grid=grid1, side=1, axis=0)
indices2 = g_f.get_boundary_indices(grid=grid2, side=1, axis=0)
# to get the geometry data, we need another instance in pyoverture
# the geometry class
# for both grids
gdata1 = Geometry(grid=grid1, cg=g_f)
gdata2 = Geometry(grid=grid2, cg=g_f)
# get coordinates
vert_int_f1 = gdata1.get_vertices(indices1)
vert_int_f2 = gdata2.get_vertices(indices2)

In [3]:
# now, in order to get the type of different grids, we need
# a special array in Overture CompositeGrids called mask
# PyOverture defines global tags for using with mask arrays
# 1. FLAG_DISCRETIZATION
# 2. FLAG_INTERPOLATION
# 3. FLAG_INTERPOLATION
# 4. FLAG_UNUSED
# 5. FLAG_UNKNOWN
mask1 = g_f.get_mask(grid1, indices1)
mask2 = g_f.get_mask(grid2, indices2)

In [4]:
# PyOverture has predefined tags for different kinds of grid points
# we need to get the union of solutions points
verts_union = []
count = 0
for k in range(indices1.size(2)):
    for j in range(indices1.size(1)):
        for i in range(indices1.size(0)):
            if mask1[count] == FLAG_DISCRETIZATION:
                verts_union.append([vert_int_f1[count][0], vert_int_f1[count][1], 0.0])
            count = count + 1

In [5]:
# Do the same for grid2
count = 0
for k in range(indices2.size(2)):
    for j in range(indices2.size(1)):
        for i in range(indices2.size(0)):
            if mask2[count] == FLAG_DISCRETIZATION:
                verts_union.append([vert_int_f2[count][0], vert_int_f2[count][1], 0.0])
            count = count + 1
verts_union = np.asarray(verts_union)

In [6]:
vert_int_s, conn_int_s, _, phy_data, _ = read('inputs/solid_tri.msh')

# get the union of node ids on boundary of tag 5
temp = set()
for i in range(conn_int_s['line'].shape[0]):
    if phy_data['line']['gmsh:physical'][i] == 5:
        temp |= set(list(conn_int_s['line'][i]))

# make a list and table
temp = list(temp)
temp.sort()
vert_int_s = vert_int_s[temp] # vertices
table = {}
for i in range(len(temp)):
    table[temp[i]] = i
temp = []
for i in range(conn_int_s['line'].shape[0]):
    if phy_data['line']['gmsh:physical'][i] == 5:
        temp.append([])
        for j in conn_int_s['line'][i]:
            temp[-1].append(table[j])
conn_int_s = temp

In [7]:
# convert integers to uint64 for solid, this is for moab
conn_int_s = np.array(conn_int_s, dtype=np.uint64)

# also, convert fluid vertices to 3D
temp = np.zeros(shape=(vert_int_f1.shape[0], 3))
temp[:, 0:2] = vert_int_f1
vert_int_f1 = temp
temp = np.zeros(shape=(vert_int_f2.shape[0], 3))
temp[:, 0:2] = vert_int_f2
vert_int_f2 = temp

In [8]:
# fluid interface meshdb
mdb_int_f1 = Core()
vert_int_f1_range = mdb_int_f1.create_vertices(vert_int_f1.flatten())
mdb_int_f2 = Core()
vert_int_f2_range = mdb_int_f2.create_vertices(vert_int_f2.flatten())
mdb_int_fu = Core()
vert_int_union = mdb_int_fu.create_vertices(verts_union.flatten())

# tags, where T for temperature and F for flux
tag_int_f1 = mdb_int_f1.tag_get_handle(
    'intTf1',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
T_int_f1 = np.zeros(vert_int_f1.shape[0])
mdb_int_f1.tag_set_data(tag_int_f1, vert_int_f1_range, T_int_f1)
flux_tag_f1 = mdb_int_f1.tag_get_handle(
    'intFf1', size=1, tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE, create_if_missing=True)
flux_int_f1 = np.zeros(vert_int_f1.shape[0])
mdb_int_f1.tag_set_data(flux_tag_f1, vert_int_f1_range, flux_int_f1)

In [9]:
# tags, where T for temperature and F for flux
tag_int_f2 = mdb_int_f2.tag_get_handle(
    'intTf2',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
T_int_f2 = np.zeros(vert_int_f2.shape[0])
mdb_int_f2.tag_set_data(tag_int_f2, vert_int_f2_range, T_int_f2)
flux_tag_f2 = mdb_int_f2.tag_get_handle(
    'intFf2', size=1, tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE, create_if_missing=True)
flux_int_f2 = np.zeros(vert_int_f2.shape[0])
mdb_int_f2.tag_set_data(flux_tag_f2, vert_int_f2_range, flux_int_f2)

In [10]:
# tags, where T for temperature and F for flux
tag_inf_fu = mdb_int_fu.tag_get_handle(
    'intTfu',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
T_int_fu = np.zeros(verts_union.shape[0])
mdb_int_fu.tag_set_data(tag_inf_fu, vert_int_union, T_int_fu)
flux_tag_fu = mdb_int_fu.tag_get_handle(
    'intFfu', size=1, tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE, create_if_missing=True)
flux_int_fu = np.zeros(verts_union.shape[0])
mdb_int_fu.tag_set_data(flux_tag_fu, vert_int_union, flux_int_fu)

In [11]:
# solid interface meshdb
mdb_int_s = Core()
vert_int_s_range = mdb_int_s.create_vertices(vert_int_s.flatten())
conn_int_s[:] += vert_int_s_range[0]
mdb_int_s.create_elements(mt.MBEDGE, conn_int_s)

# tags, where T for temperature and F for flux
tag_int_s = mdb_int_s.tag_get_handle(
    'intTs',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
T_int_s = np.zeros(vert_int_s.shape[0])
mdb_int_s.tag_set_data(tag_int_s, vert_int_s_range, T_int_s)
flux_tag_s = mdb_int_s.tag_get_handle(
    'intFs', size=1, tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE, create_if_missing=True)
flux_int_s = np.zeros(vert_int_s.shape[0])
mdb_int_s.tag_set_data(flux_tag_s, vert_int_s_range, flux_int_s)

In [12]:
# configure mappers, DTK uses RBFs, the radius should not be small
# otherwise, you may not have enough neighborhood for the fitting
radius = 0.4

# create solid manager
dtk_mnr_s = DTK2MoabManager(mdb_int_s)
dtk_mnr_s.register_tags(['intTs', 'intFs'])

# create fluid manager 1
dtk_mnr_f1 = DTK2MoabManager(mdb_int_f1)
dtk_mnr_f1.register_tags(['intTf1', 'intFf1'])

# create fluid manager 2
dtk_mnr_f2 = DTK2MoabManager(mdb_int_f2)
dtk_mnr_f2.register_tags(['intTf2', 'intFf2'])

# create fluid manager union
dtk_mnr_fu = DTK2MoabManager(mdb_int_fu)
dtk_mnr_fu.register_tags(['intTfu', 'intFfu'])

# solid temperature to fluid mapper 1
sT2f_mapper1 = DTK2Mapper(source=dtk_mnr_s, target=dtk_mnr_f1, r=radius)
sT2f_mapper1.register_coupled_tags('intTs', 'intTf1')

#solid temperature to fluid mapper 2
sT2f_mapper2 = DTK2Mapper(source=dtk_mnr_s, target=dtk_mnr_f2, r=radius)
sT2f_mapper2.register_coupled_tags('intTs', 'intTf2')

# fluid flux to solid mapper
fF2s_mapper = DTK2Mapper(source=dtk_mnr_fu, target=dtk_mnr_s, r=radius)
fF2s_mapper.register_coupled_tags('intFfu', 'intFs')

In [13]:
# a union solution helper
def union_sols(ind1, ind2, msk1, msk2, arr1, arr2, oarr):
    count = 0
    count2 = 0
    for k in range(ind1.size(2)):
        for j in range(ind1.size(1)):
            for i in range(ind1.size(0)):
                if msk1[count] == FLAG_DISCRETIZATION:
                    oarr[count2] = arr1[count]
                    count2 = count2+1
                count = count+1
    
    count = 0
    for k in range(ind2.size(2)):
        for j in range(ind2.size(1)):
            for i in range(ind2.size(0)):
                if msk2[count] == FLAG_DISCRETIZATION:
                    oarr[count2] = arr2[count]
                    count2 = count2+1
                count = count+1
    return oarr

In [14]:
# FEM solver
kappa_s = 0.7
specific_heat = 0.485
solver_s = HeatSolver(kappa=kappa_s, c=specific_heat)
solver_s.load_mesh('inputs/solid_tri.msh')
solver_s.set_initial_condition(0.0)
solver_s.set_boundary(2, 0.0, boundary_type='Neumann') # bot
solver_s.set_boundary(3, 1.0) # right
solver_s.set_boundary(4, 0.0, boundary_type='Neumann') # top
solver_s.set_boundary(5, 0.0, boundary_type='Neumann') # interface
solver_s.set_source(0.0) # forcing term

# init
solver_s.init_solver()

solver_s.output_file = 'outputs/solid.pvd'

In [15]:
# fluid solver
solver_f = Cgins()
solver_f.read_cmd('inputs/fluidCgins.cmd')
solver_f.init_grid()
solver_f.init_temperature_interface(grid=grid1, side=1, axis=0)  # must do this!
solver_f.init_temperature_interface(grid=grid2, side=1, axis=0)  # must do this!
solver_f.init_solver(debug=0)

kappa_f = solver_f.get_thermal_conductivity()
assert kappa_f > 0.0

In [16]:
inall1 = g_f.get_indices(grid=grid1)
inall2 = g_f.get_indices(grid=grid2)
v1 = gdata1.get_vertices(inall1)
temp = np.zeros(shape=(v1.shape[0], 3), dtype=np.float64)
temp[:, 0:2] = v1
v1 = temp
v2 = gdata2.get_vertices(inall2)
temp = np.zeros(shape=(v2.shape[0], 3), dtype=np.float64)
temp[:, 0:2] = v2
v2 = temp
mskall1 = g_f.get_mask(grid1, inall1)
mskall2 = g_f.get_mask(grid2, inall2)
conn1 = np.array(inall1.generate_volume_connectivity(2), dtype=np.uint64)
conn2 = np.array(inall2.generate_volume_connectivity(2), dtype=np.uint64)
# verts = []
# count = 0
# for k in range(inall1.size(2)):
#     for j in range(inall1.size(1)):
#         for i in range(inall1.size(0)):
#             if mskall1[count] == FLAG_DISCRETIZATION:
#                 verts.append([v1[count][0], v1[count][1], 0.0])
#             count = count + 1
# count = 0
# for k in range(inall2.size(2)):
#     for j in range(inall2.size(1)):
#         for i in range(inall2.size(0)):
#             if mskall2[count] == FLAG_DISCRETIZATION:
#                 verts.append([v2[count][0], v2[count][1], 0.0])
#             count = count + 1
# verts = np.asarray(verts)
mdbF = Core()
v1range = mdbF.create_vertices(v1.flatten())
v2range = mdbF.create_vertices(v2.flatten())
conn1[:] += v1range[0]
conn2[:] += v2range[0]
elem1 = mdbF.create_elements(mt.MBQUAD, conn1)
elem2 = mdbF.create_elements(mt.MBQUAD, conn2)
patch1 = mdbF.create_meshset()
mdbF.add_entities(patch1, v1range)
mdbF.add_entities(patch1, elem1)
patch2 = mdbF.create_meshset()
mdbF.add_entities(patch2, v2range)
mdbF.add_entities(patch2, elem2)

In [17]:
tempF = mdbF.tag_get_handle(
    'fT',
    size=1,
    tag_type=mt.MB_TYPE_DOUBLE,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
mtag = mdbF.tag_get_handle(
    'mask',
    size=1,
    tag_type=mt.MB_TYPE_INTEGER,
    storage_type=mt.MB_TAG_DENSE,
    create_if_missing=True)
mdbF.tag_set_data(mtag, v1range, mskall1)
mdbF.tag_set_data(mtag, v2range, mskall2)

In [18]:
# coupling
Tfinal = 1.0
N = 200
dt = Tfinal / N
t = 0.0

In [19]:
for i in range(N):
    t += dt

    ##########
    # FLUID I
    ##########
    
    # advance, if there is still steps left, advance a step
    # note that the small dt is automatically updated.
    solver_f.advance(t, dt)
    fT1 = solver_f.get_temperature(grid1, inall1)
    fT2 = solver_f.get_temperature(grid2, inall2)
    mdbF.tag_set_data(tempF, v1range, fT1)
    mdbF.tag_set_data(tempF, v2range, fT2)
    mdbF.write_file('outputs/fluid%i.vtk' % i, output_sets=[patch1, patch2])
        
            
    # get fluid interface heat flux in grid1
    flux_int_f1[:] = solver_f.get_normal_heat_flux_interface(grid=grid1, side=1, axis=0)
    # scale
    flux_int_f1[:] *= kappa_f
    # get fluid interface heat flux in grid2
    flux_int_f2[:] = solver_f.get_normal_heat_flux_interface(grid=grid2, side=1, axis=0)
    # scale
    flux_int_f2[:] *= kappa_f
    # get the union
    flux_int_fu[:] = union_sols(indices1, indices2, mask1, mask2, flux_int_f1, flux_int_f2, flux_int_fu)

    ###########
    # MAPPING
    ###########
    
    # put to meshdb
    mdb_int_fu.tag_set_data(flux_tag_fu, vert_int_union, flux_int_fu)
    # transfer
    fF2s_mapper.apply('intFfu', 'intFs')

    ##########
    # SOLID
    ##########
    
    # get data from meshdb
    flux_int_s[:] = mdb_int_s.tag_get_data(flux_tag_s, vert_int_s_range, True)
    # solid update the heat flux interface, remind the tag is 5
    # cht_interface indicates that this value is given by CHT interface
    # so internally, it will take care of scaling and normal directions
    solver_s.update_flux_interface(5, flux_int_s, cht_interface=True)
    # advance
    solver_s.advance(dt, t)
    # get temperature
    T_int_s[:] = solver_s.get_solutions(where='boundary', tag=5)

    ###########
    # MAPPING
    ###########
    
    # put to meshdb
    mdb_int_s.tag_set_data(tag_int_s, vert_int_s_range, T_int_s)
    # transfer to grid1
    sT2f_mapper1.apply('intTs', 'intTf1')
    sT2f_mapper2.apply('intTs', 'intTf2')

    ##########
    # FLUID II
    ##########
    
    # get data from meshdb for grid1
    T_int_f1[:] = mdb_int_f1.tag_get_data(tag_int_f1, vert_int_f1_range, True)
    # fluid updating the temperature boundary condition
    solver_f.put_temperature_interface(grid=grid1, side=1, axis=0, data=T_int_f1)
    # get data from meshdb for grid1
    T_int_f2[:] = mdb_int_f2.tag_get_data(tag_int_f2, vert_int_f2_range, True)
    # fluid updating the temperature boundary condition
    solver_f.put_temperature_interface(grid=grid2, side=1, axis=0, data=T_int_f2)
    
    ##########
    # OUTPUTS
    ##########

    solver_s.write()

In [20]:
# write two patches separately for visualization
mdbF.write_file('outputs/fp1.vtk', [patch1])
mdbF.write_file('outputs/fp2.vtk', [patch2])